In [1]:
# load file

import json
with open('wien_bar-pub-cafe_walk_distances.json') as f:
    data = json.load(f)


In [2]:
print(data['nodes'][0])
print(data['edges'][0])

{'id': 566660740, 'lat': '48.2120210', 'lon': '16.3723092', 'addr:city': 'Wien', 'addr:country': 'AT', 'addr:housenumber': '11', 'addr:postcode': '1010', 'addr:street': 'Sterngasse', 'amenity': 'bar', 'name': 'Funky Monkey', 'opening_hours': 'Th 20:00-02:00;Fr-Sa 20:00-04:00', 'website': 'http://www.funkymonkey.at', 'wheelchair': 'no'}
{'from': 566660740, 'to': 566660785, 'distance': 109}


In [3]:
nodes_dict = {}
for n in data['nodes']:
    nodes_dict[n['id']]=n

In [4]:
edge_distance = {}
for e in data['edges']:
    edge_distance[(e['from'], e['to'])] = e['distance']
    edge_distance[(e['to'], e['from'])] = e['distance']

In [5]:
bezirk_nodes_dict = {}

for i in range(1,24):
    plz="1{:02d}0".format(i)
    bezirk_nodes = []
    for n in data['nodes']:
        if n['addr:postcode'] == plz:
            bezirk_nodes.append(n['id'])
    bezirk_nodes_dict[i] = bezirk_nodes


In [6]:
from random import choice

def random_pubs():
    pubs_choices = []
    for i in range(1,24):
        pubs_choices.append(choice(bezirk_nodes_dict[i]))
    return pubs_choices    

In [7]:
import networkx as nx

def best_route(pubs_selection):
    # unidirected graph
    g = nx.Graph()
    for id1 in range(len(pubs_selection)):
        for id2 in range(id1+1, len(pubs_selection)):
               g.add_edge(pubs_selection[id1],pubs_selection[id2], weight=edge_distance[(pubs_selection[id1],pubs_selection[id2])])
    
    best_route = nx.approximation.traveling_salesman_problem(g, weight='weight', nodes=pubs_selection, cycle=False)
    return best_route


In [8]:
def total_distance(route):
    distance_total = 0
    for i in range(1,len(route)):
        distance_total += edge_distance[(route[i-1], route[i])]
    return distance_total

Simple GA inpired by https://www.youtube.com/watch?v=nhT56blfRpE

In [9]:
def generate_population(size):
    return [random_pubs() for _ in range(size)]

In [10]:
from random import choices, randint, randrange, random

def single_point_crossover(a, b):
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]

In [11]:
def mutation(genome, num: int = 1, probability: float = 0.6):
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else choice(bezirk_nodes_dict[index+1])
    return genome

In [12]:
def fitness_func(pubs_selection):
    fitness =  total_distance(best_route(pubs_selection))
    return fitness

In [13]:
def population_fitness(population):
    return sum([fitness_func(genome) for genome in population])

In [14]:
def selection_pair(population):
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )

In [15]:
def sort_population(population):
    return sorted(population, key=fitness_func, reverse=True)

In [17]:
import folium

def print_route(route):
    node0 = nodes_dict[route[0]]
    m = folium.Map(location=[float(node0['lat']),float(node0['lon'])])

    #draw nodes
    for i in range(len(route)):
        node = nodes_dict[route[i]]
        folium.Marker([float(node['lat']),float(node['lon'])], tooltip=f"<i>{i+1}: {node['name']} {node['addr:postcode']}</i>").add_to(m)

    # draw edges
    for i in range(1,len(route)):
        node_previous = route[i-1]
        node_current = route[i]
        loc=[(float(nodes_dict[node_previous]['lat']), float(nodes_dict[node_previous]['lon'])),(float(nodes_dict[node_current]['lat']), float(nodes_dict[node_current]['lon'])) ]
        duration = edge_distance[(node_previous, node_current)]
        folium.PolyLine(loc, color='red', weight=5,opacity=0.8, tooltip=duration).add_to(m)    


    return m

In [18]:
def run_evolution(
        fitness_limit: int = 50000,
        population_size: int = 10,
        generation_limit: int = 100,
        mutation_num: int = 1,  
        mutation_probability: float = 0.5):
    
    population = generate_population(population_size)

    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=False)

        fitness = fitness_func(population[0])
        print(f"generation {i} - distance: {fitness}m")
        if fitness <= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_pair(population)
            offspring_a, offspring_b = single_point_crossover(parents[0], parents[1])
            offspring_a = mutation(offspring_a, mutation_num, mutation_probability)
            offspring_b = mutation(offspring_b, mutation_num, mutation_probability)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [19]:
from contextlib import contextmanager
import time


@contextmanager
def timer():
    start = time.time()
    yield
    end = time.time()
    print(f"Elapsed Time: {(end - start)}s")

In [20]:
# find best nodes

with timer():
    population, generations = run_evolution(fitness_limit=35000, generation_limit=1500, population_size = 10, mutation_num=1, mutation_probability=0.50)

print_route(best_route(population[0]))

generation 0 - distance: 58636m
generation 1 - distance: 58636m
generation 2 - distance: 58636m
generation 3 - distance: 57721m
generation 4 - distance: 56309m
generation 5 - distance: 56309m
generation 6 - distance: 55492m
generation 7 - distance: 55492m
generation 8 - distance: 55492m
generation 9 - distance: 55042m
generation 10 - distance: 52116m
generation 11 - distance: 52116m
generation 12 - distance: 52116m
generation 13 - distance: 52116m
generation 14 - distance: 51273m
generation 15 - distance: 51273m
generation 16 - distance: 51273m
generation 17 - distance: 51273m
generation 18 - distance: 51273m
generation 19 - distance: 51273m
generation 20 - distance: 51273m
generation 21 - distance: 51273m
generation 22 - distance: 51193m
generation 23 - distance: 51193m
generation 24 - distance: 51129m
generation 25 - distance: 51129m
generation 26 - distance: 51129m
generation 27 - distance: 51129m
generation 28 - distance: 51129m
generation 29 - distance: 51129m
generation 30 - dist

generation 245 - distance: 43605m
generation 246 - distance: 43605m
generation 247 - distance: 43605m
generation 248 - distance: 43605m
generation 249 - distance: 43604m
generation 250 - distance: 43604m
generation 251 - distance: 43604m
generation 252 - distance: 43604m
generation 253 - distance: 43604m
generation 254 - distance: 43604m
generation 255 - distance: 43604m
generation 256 - distance: 43604m
generation 257 - distance: 43604m
generation 258 - distance: 43082m
generation 259 - distance: 43082m
generation 260 - distance: 43082m
generation 261 - distance: 43082m
generation 262 - distance: 43082m
generation 263 - distance: 43060m
generation 264 - distance: 43060m
generation 265 - distance: 43060m
generation 266 - distance: 43060m
generation 267 - distance: 43060m
generation 268 - distance: 43060m
generation 269 - distance: 43060m
generation 270 - distance: 43060m
generation 271 - distance: 43060m
generation 272 - distance: 43060m
generation 273 - distance: 43060m
generation 274

generation 486 - distance: 40850m
generation 487 - distance: 40850m
generation 488 - distance: 40850m
generation 489 - distance: 40850m
generation 490 - distance: 40850m
generation 491 - distance: 40850m
generation 492 - distance: 40850m
generation 493 - distance: 40850m
generation 494 - distance: 40850m
generation 495 - distance: 40850m
generation 496 - distance: 40850m
generation 497 - distance: 40850m
generation 498 - distance: 40850m
generation 499 - distance: 40850m
generation 500 - distance: 40850m
generation 501 - distance: 40850m
generation 502 - distance: 40850m
generation 503 - distance: 40850m
generation 504 - distance: 40850m
generation 505 - distance: 40850m
generation 506 - distance: 40850m
generation 507 - distance: 40850m
generation 508 - distance: 40850m
generation 509 - distance: 40850m
generation 510 - distance: 40850m
generation 511 - distance: 40850m
generation 512 - distance: 40850m
generation 513 - distance: 40850m
generation 514 - distance: 40850m
generation 515

generation 727 - distance: 39060m
generation 728 - distance: 39060m
generation 729 - distance: 39060m
generation 730 - distance: 39060m
generation 731 - distance: 39060m
generation 732 - distance: 39060m
generation 733 - distance: 39060m
generation 734 - distance: 39060m
generation 735 - distance: 39060m
generation 736 - distance: 39060m
generation 737 - distance: 39060m
generation 738 - distance: 39060m
generation 739 - distance: 39060m
generation 740 - distance: 39060m
generation 741 - distance: 39060m
generation 742 - distance: 39060m
generation 743 - distance: 39060m
generation 744 - distance: 39060m
generation 745 - distance: 39060m
generation 746 - distance: 39060m
generation 747 - distance: 39060m
generation 748 - distance: 39060m
generation 749 - distance: 39060m
generation 750 - distance: 39060m
generation 751 - distance: 39060m
generation 752 - distance: 39060m
generation 753 - distance: 39060m
generation 754 - distance: 39060m
generation 755 - distance: 39060m
generation 756

generation 968 - distance: 38816m
generation 969 - distance: 38816m
generation 970 - distance: 38816m
generation 971 - distance: 38816m
generation 972 - distance: 38816m
generation 973 - distance: 38816m
generation 974 - distance: 38816m
generation 975 - distance: 38816m
generation 976 - distance: 38816m
generation 977 - distance: 38816m
generation 978 - distance: 38816m
generation 979 - distance: 38816m
generation 980 - distance: 38816m
generation 981 - distance: 38816m
generation 982 - distance: 38816m
generation 983 - distance: 38816m
generation 984 - distance: 38816m
generation 985 - distance: 38816m
generation 986 - distance: 38816m
generation 987 - distance: 38816m
generation 988 - distance: 38816m
generation 989 - distance: 38816m
generation 990 - distance: 38816m
generation 991 - distance: 38816m
generation 992 - distance: 38816m
generation 993 - distance: 38816m
generation 994 - distance: 38816m
generation 995 - distance: 38816m
generation 996 - distance: 38816m
generation 997

generation 1203 - distance: 38398m
generation 1204 - distance: 38398m
generation 1205 - distance: 38398m
generation 1206 - distance: 38398m
generation 1207 - distance: 38398m
generation 1208 - distance: 38398m
generation 1209 - distance: 38398m
generation 1210 - distance: 38398m
generation 1211 - distance: 38398m
generation 1212 - distance: 38398m
generation 1213 - distance: 38398m
generation 1214 - distance: 38398m
generation 1215 - distance: 38398m
generation 1216 - distance: 38398m
generation 1217 - distance: 38398m
generation 1218 - distance: 38398m
generation 1219 - distance: 38398m
generation 1220 - distance: 38398m
generation 1221 - distance: 38398m
generation 1222 - distance: 38398m
generation 1223 - distance: 38398m
generation 1224 - distance: 38398m
generation 1225 - distance: 38398m
generation 1226 - distance: 38398m
generation 1227 - distance: 38398m
generation 1228 - distance: 38398m
generation 1229 - distance: 38398m
generation 1230 - distance: 38398m
generation 1231 - di

generation 1438 - distance: 38396m
generation 1439 - distance: 38396m
generation 1440 - distance: 38396m
generation 1441 - distance: 38396m
generation 1442 - distance: 38396m
generation 1443 - distance: 38396m
generation 1444 - distance: 38396m
generation 1445 - distance: 38396m
generation 1446 - distance: 38396m
generation 1447 - distance: 38396m
generation 1448 - distance: 38396m
generation 1449 - distance: 38396m
generation 1450 - distance: 38396m
generation 1451 - distance: 38396m
generation 1452 - distance: 38396m
generation 1453 - distance: 38396m
generation 1454 - distance: 38396m
generation 1455 - distance: 38396m
generation 1456 - distance: 38396m
generation 1457 - distance: 38396m
generation 1458 - distance: 38396m
generation 1459 - distance: 38396m
generation 1460 - distance: 38396m
generation 1461 - distance: 38396m
generation 1462 - distance: 38396m
generation 1463 - distance: 38396m
generation 1464 - distance: 38396m
generation 1465 - distance: 38396m
generation 1466 - di

In [21]:
br1 = population[0]
br1

[319735103,
 3456346364,
 1362753598,
 423005412,
 1439072576,
 9575040983,
 325140051,
 331584946,
 1372123916,
 8812381966,
 2538201846,
 9282066486,
 2458558771,
 7792720511,
 289642464,
 2647300062,
 355211469,
 306005705,
 371798859,
 6184632917,
 3390316821,
 1365701433,
 1022767606]

In [ ]:
print_route(best_route(br1))

In [ ]:
#random search

best_found_route = []
best_found_distance = 100000
for i in range(1000):
    route = best_route(random_pubs())
    distance = total_distance(route)
    if distance < best_found_distance:
        best_found_distance = distance
        best_found_route = route

best_found_distance